In [1]:
!pip install pandas
import numpy as np
import pandas as pd
from numpy.linalg import norm

In [18]:
# change the file path here
file_path_base = r""
file_path_info = file_path_base + "id_information_mmsr.tsv" 
file_path_word2vec = file_path_base + "id_lyrics_word2vec_mmsr.tsv"
file_path_tfidf = file_path_base + "id_lyrics_tf-idf_mmsr.tsv"
file_path_bert = file_path_base + "id_lyrics_bert_mmsr.tsv"

df_info = pd.read_table(file_path_info)
df_word2vec = pd.read_table(file_path_word2vec)
df_tfidf = pd.read_table(file_path_tfidf)
df_bert = pd.read_table(file_path_bert)

df_info

,id,artist,song,album_name
0,01Yfj2T3YTwJ1Yfy,We As Human,Take The Bullets Away (feat. Lacey Sturm),We As Human
1,01gyRHLquwXDlhkO,The Notorious B.I.G.,Somebody's Gotta Die,Life After Death (Remastered Edition)
2,01rMxQv6vhyE1oQX,Against the Current,Chasing Ghosts,In Our Bones
3,02RGE9FNH65RtMS7,Barthezz,Infected,Trance - The Early Years (1997-2002)
4,02ZnlCGZEbkfCDxo,Laura Pausini,Tra Te E Il Mare,The Best of Laura Pausini - E Ritorno Da Te
...,...,...,...,...
10090,zyzILCQvVeUFIINi,Crowded House,When You Come,Temple Of Low Men
10091,zzgS4ZqyswamEWNj,Britney Spears,My Only Wish (This Year),Platinum Christmas
10092,zzoFYDMlqU1X2zz1,Thundercat,DUI,Drunk
10093,zzpkRCGA5ud8q4mv,Otis Redding,Rock Me Baby,Otis Blue


In [19]:
# Random baseline

def random_baseline(info: pd.DataFrame, title: str, artist: str, n: int) -> pd.DataFrame:
    return info.sample(n=n)[["id","artist","song"]]

random_baseline(df_info, "Jingle Bells", "Frank Sinatra", 10)

,id,artist,song
7427,jqdOn7hVCRxa1Veb,Death,Bite the Pain
3409,KyUXMt1bUkNEExrO,Ween,Loving U Thru It All
169,0vTCI3MyJ8ySMrWj,Marilyn Manson,You And Me And The Devil Makes 3
1490,8tUUOUHWyVgoZOat,Delegation,Put a Little Love On Me
8561,qVLc7f00yBJUcWm1,Janelle Monáe,Tightrope
7654,lCji3lTQfY0hxwqD,Mayhem,Deconsecrate
3080,Il4JempJ0dxcVphD,Mallu Magalhães,Te Acho Tão Bonito
8152,o5S6SQ69DTnMZGQ9,Beyond the Black,Numb
3137,J7ks4R4hI0gmCoQK,Kansas,Sparks of the Tempest
5920,aeFBjUvKSmIAXICu,Shania Twain,It Only Hurts When I'm Breathing


In [20]:
random_baseline(df_info, "Shape of You", "Ed Sheeran", 10)

,id,artist,song
8337,pBS8YUMxWpvLmrfI,Oasis,The Masterplan
2996,IC70veAhCXIpesFV,Sepultura,Lookaway
20,09mWmQm7ixF4cCjx,The Word Alive,Life Cycles
4707,Sp3mZLAsLxAKPry5,The Sound,The Fire
5108,VLI1rXN8FFfHDBkJ,Rites of Spring,Other Way Around
6323,dIbdUy8MNt9UItbs,Paloma Faith,Just Be
8507,qDvTQEvu6ZCCMdk6,Dark Funeral,Unchain My Soul
4665,SYVuCJFFlsGfr2mZ,Akon,Beautiful
7300,iz4VGpeuKwnDestf,BTS,A Supplementary Story: You Never Walk Alone
5353,WvfI9nwOhK7dOlpJ,JP Nataf,Mon Ami D'En Haut


In [21]:
random_baseline(df_info, "Natural", "Imagine Dragons", 10)

,id,artist,song
8728,ra1IZ3joqDs7XrGI,The Clash,Red Angel Dragnet
327,1mPpPt4RxMkZrSBT,The Postal Service,Sleeping In
2075,CVirHSstAw0VXT6n,Big Time Rush,Oh Yeah
6256,cqDVwhm4JVsQSzae,Raul Seixas,Novo Aeon
3928,OCM57tsHAvVxlaGM,Grandson,Despicable
4367,Qln3KNznkzmJVv68,Erasmo Carlos,Mundo Deserto
9882,yZPLE6qF7hqSAzNE,Courtney Barnett,Lance Jr.
2409,EaWFlVMGu79goLHv,A Day to Remember,You Should Have Killed Me When You Had the Chance
2573,FeWT4cWDN1bPnEuY,Paramore,Miracle
4772,TBH4dxkeSVvTsSnV,Camila,Coleccionista De Canciones


In [22]:
def cos_sim(query: [int], target: [int]) -> int:
    return np.dot(query,target)/(norm(query)*norm(target))

def euc_sim(query: [int], target: [int]) -> int:
    return 1/(1+norm(query-target))
  

def text_based_retrieval(info: pd.DataFrame, feature: pd.DataFrame, title: str, artist: str, n: int, sim_func = cos_sim) -> pd.DataFrame:
    feature_no_id = feature.drop(columns="id") # drop id column for similarity measurement
    query_id = info[(info["artist"] == artist) & (info["song"] == title)]["id"].values[0] # search for query song in info
    query = feature[feature["id"]==query_id].drop(columns="id").values[0] # get feature vector for query song
    sims = [sim_func(query, target) for target in feature_no_id.values] # compute similarity between query and each target song
    feature["sim"] = sims # add similiarity column to feature
    info_feature = pd.merge(info, feature[["id", "sim"]], on="id") # merge feature with info by id
    info_feature_sorted = info_feature.sort_values(by=["sim"], ascending=False) # sort by similiartiy in descending order
    return info_feature_sorted[["id","artist","song","sim"]][1:n+1] # skips the first row, because it is the query track

In [23]:
# Text-based(cos-sim, tf-idf)
text_based_retrieval(df_info, df_tfidf, "Jingle Bells", "Frank Sinatra", 10)

,id,artist,song,sim
9160,u8bj2RyzoYZ99dWB,Gwen Stefani,Jingle Bells,0.963528
6081,blZ9zSQBqOMxcPhN,Vanessa Carlton,Hear the Bells,0.601615
6540,eU30OjpKt9zzV6R6,Lil Xan,Saved by the Bell,0.526900
3249,JtwyzoBa2N48HsHo,The Beatles,Michelle,0.507277
9709,xSbRgzIgvXuoeLpL,The Black Heart Procession,Your Church Is Red,0.457886
7142,hxGDHGbn3KTlfp6d,Metallica,For Whom The Bell Tolls - Remastered,0.407539
3130,J4onmjAmjdnYYbpX,The Faint,Southern Belles In London Sing,0.372241
7878,mTbTSXakQDcIH7MK,Dire Straits,Portobello Belle,0.365819
4116,PK2m4Mc7MnPT6az8,Gregory and the Hawk,Voice Like a Bell,0.331044
8397,pZijIGqH2GBH2U3X,The Free Design,Kites Are Fun,0.317380


In [24]:
text_based_retrieval(df_info, df_tfidf, "Shape of You", "Ed Sheeran", 10)

,id,artist,song,sim
7095,hk8cDn49nRGqyAFI,Christina Aguilera,Ven Conmigo (Solamente Tú),0.581465
7727,lYWQPI2TFoh8nSTC,Blur,Tender,0.551355
10012,zPnDmTeGC4vikUDP,Natalia Lafourcade,No Viniste,0.499672
5782,ZmdJLhQm8gtsOkjW,Clap Your Hands Say Yeah,Over and Over Again (Lost and Found),0.466910
292,1YOagKsFqTS8BDK2,Zara Larsson,I Would Like,0.463639
2405,EZJM2B4piUzzIJPo,Vanessa Hudgens,Come Back to Me,0.459581
10023,zT3GPtC2T6AyfG9b,Suicide,Cheree,0.423330
2272,DdVnX8c2v5NJtBu4,Aventura,Cuando volveras,0.418954
2634,G2otV6WAmea6VB1f,Steps,Heartbeat,0.416584
9012,tM6vR5sdaje8tjmU,Jenny Hval,Female Vampire,0.416305


In [25]:
text_based_retrieval(df_info, df_tfidf, "Natural", "Imagine Dragons", 10)

,id,artist,song,sim
5124,VTiQufVGka4dXkST,Napalm Death,Hierarchies,0.544006
9428,vjkaT5dKknSZY00H,Vanessa Paradis,Natural High,0.512680
6744,fcqqxxck3btykgKT,Woods of Ypres,Keeper of the Ledger,0.503596
3501,LXg9nms2tgD14h3B,Papa Roach,Blood Brothers,0.496838
4050,OvWDkevn5dCLHvrW,Turnover,Super Natural,0.436332
7444,jxocCDUjcOYHAoXx,Gilbert O'Sullivan,Alone Again (Naturally),0.425614
9847,yMRvpBNGnVzi5N6s,Carole King,(You Make Me Feel Like) A Natural Woman,0.410265
2558,FWgz1hVIq59Ve6SY,PJ Harvey,On Battleship Hill,0.328914
293,1YUiDKKSD7GIcXyx,Forfun,Cósmica,0.325034
9127,tyH1H0Aoy6l2E3bN,Wye Oak,It Was Not Natural,0.317386


In [26]:
# Text-based(cos-sim, bert)
text_based_retrieval(df_info, df_bert, "Jingle Bells", "Frank Sinatra", 10)

,id,artist,song,sim
9160,u8bj2RyzoYZ99dWB,Gwen Stefani,Jingle Bells,0.951122
6427,doTmvQlJVL1JRO4V,Robert Johnson,Hellhound On My Trail,0.662801
4621,SJZTstFdLSYvbRAi,Change,The Glow of Love,0.623144
5661,YzXWwWKeFMKNgkU7,Hot Chip,"You Ride, We Ride, In My Ride",0.617038
1585,9ScGeeaW8XcxgePd,Kelly Clarkson,Every Christmas,0.603126
9989,zHozLx4GhJsG7xLJ,Cyndi Lauper,Christmas Conga,0.598624
3442,LArarDy0SyTJDoIZ,Eric Clapton,Circus,0.597165
5927,afmSDk2caOd8CCfx,B*Witched,Rollercoaster,0.592372
6176,cItkoIyGIr9LrIJS,Beirut,Elephant Gun,0.590946
5907,aYfhVF6MlwbLPm0i,Rihanna,We Ride,0.590712


In [27]:
text_based_retrieval(df_info, df_bert, "Shape of You", "Ed Sheeran", 10)

,id,artist,song,sim
8951,sxnkfQ7yHuBOZFjJ,Girls Aloud,Jump,0.837990
3693,MhYLWu5NwwmAzrnq,Mary J. Blige,I'm In Love,0.837520
3315,KJyVTXfwdjUGtj5m,Boyzone,Love Is A Hurricane,0.832975
3190,JSW7OaMUVh17yN8J,Michael Jackson,The Way You Make Me Feel,0.831327
2141,CqHIvWSho0esW9se,Crayon Pop,Bing Bing,0.831041
4368,QlwiTDUXdw6TlKMf,Big Mountain,Baby I Love Your Way,0.830039
10074,zu435IwGtNr9JjMF,Teddy Pendergrass,Turn off the Lights,0.823441
1504,8xFD1UO8nr1qwOg6,Ed Sheeran,Thinking Out Loud,0.823009
1876,BGka5j0KKRDSzj1N,Teena Marie,Lovergirl,0.822292
6799,g1Fkyx0T1Ylcw49Z,Teena Marie,Ooo La La La,0.820115


In [28]:
text_based_retrieval(df_info, df_bert, "Natural", "Imagine Dragons", 10)

,id,artist,song,sim
1778,AiE9Cln05EjE1dOf,Within Temptation,The Heart of Everything,0.822411
5543,YCl58AvJwcQ5F7jY,Dream Theater,Learning to Live,0.822307
5753,ZctViWjWYSA62gVU,Draconian,Rivers Between Us,0.819205
3421,L2gnt8dqIkmRjefp,Fallujah,The Void Alone,0.818282
1142,6tRWuK7iWbu3wkFf,Joe McElderry,Ambitions,0.816821
6481,e6Os8czgZksMBWXw,Foster the People,Pseudologia Fantastica,0.816118
7313,j63Pu2SUnO4OASD9,Mystery Jets,Someone Purer,0.813864
5297,Wb1iIt34MSoFeLrh,Of Monsters and Men,Hunger,0.812177
4862,TnDbCfYqdBuEN1h2,My Chemical Romance,Burn Bright,0.811639
4797,TMKMcGJdBPDL2acD,David Bowie,Teenage Wildlife,0.811501


In [29]:
# Text-based(euc-sim, word2vec)
text_based_retrieval(df_info, df_word2vec, "Jingle Bells", "Frank Sinatra", 10, euc_sim)

,id,artist,song,sim
9160,u8bj2RyzoYZ99dWB,Gwen Stefani,Jingle Bells,0.931556
6464,dyPSAz5u5gGVbs9R,Nat King Cole,The Christmas Song (Merry Christmas To You),0.798056
9989,zHozLx4GhJsG7xLJ,Cyndi Lauper,Christmas Conga,0.790671
788,4gMuVWDrTgZsGrlR,Pigeon John,The Bomb,0.788332
5212,W5corTpjrk7QEawo,Faun,Walpurgisnacht,0.787523
9056,tbF2tUUkAXDzk7sZ,Kult,Celina,0.786475
4301,QPUaa2orGuoeXzwk,Little Feat,Dixie Chicken,0.786340
5072,V9weUlcyzPJ2qbRn,Caravan,Winter Wine,0.785717
2026,CCP8zIK9PsURC8VV,Ella Fitzgerald,Manhattan,0.785132
681,40FzEJFj72Vme3Kn,Led Zeppelin,Bron-Y-Aur Stomp,0.784964


In [30]:
text_based_retrieval(df_info, df_word2vec, "Shape of You", "Ed Sheeran", 10, euc_sim)

,id,artist,song,sim
3662,MUVVvDQsGV3hkwuK,Girls Aloud,Graffiti My Soul,0.874713
941,5bVqP2AsQwwTBO1v,Amy Winehouse,Moody's Mood for Love,0.870368
6743,fbjU4wiAgu6prbVW,Röyksopp,40 Years Back Come,0.867821
7188,iHAUwRKkWyLqAmUc,Nick Cave & The Bad Seeds,Anthrocene,0.867417
5487,Xnrqj9qzUNzw6WEP,Alejandro Sanz,Mi soledad y yo,0.864460
6195,cR5LYvKway60BZIk,Jackson C. Frank,Relations,0.864338
3044,IWFOSAt5uDtoXWen,Little Mix,Love Me Like You,0.864236
6725,fVIdrI2P5FzC0u8h,Sun Kil Moon,Duk Koo Kim,0.864070
7081,he2vktX5wOCvKFdh,Miguel Bosé,Si Tu No Vuelves,0.861178
7975,n74xddJ3ud7ROzLR,Brett Dennen,Ain't No Reason,0.860669


In [32]:
text_based_retrieval(df_info, df_word2vec, "Natural", "Imagine Dragons", 10, euc_sim)

,id,artist,song,sim
8043,nRwISjF6fC9mDEaG,Bright Eyes,June on the West Coast,0.836854
2897,HasvlKVkh3kNRLEg,Capsize,Pale,0.835781
8381,pTGwOCXtAhZOXPfG,Bright Eyes,Classic Cars,0.831929
4978,UYnklBBToUS2b2yW,Haikaiss,Raplord,0.830380
7782,ltxbqSGFwcdiFR8k,Architects,Alpha Omega,0.830312
4390,QvYZirxrdhrWaEOq,Entombed,Blessed Be,0.830254
1978,BvavHfrEqUNNphJD,Eminem,Believe,0.830088
2802,H2YkL2APQ7R3emwZ,Massive Attack,Risingson,0.830019
1227,7SvyWNO8okpttyo8,The Roots,Walk Alone,0.829862
1476,8oVmZB9m77Pfx1eK,The Roots,Dear God 2.0,0.829315
